In [ ]:
import fuzzingbook_utils

# Project 1

The goal of this project is to utilize mutation fuzzing to cover as much code as possible during testing.

The target is the [svglib](https://pypi.org/project/svglib/) SVG rendering library written in python.

For an easier integration the library we provide a wrapped function __parse_svg(string)__, which receives a string with the SVG content and invokes the parsing library. 

To ensure that all converted elements are correct, the wrapper function internally converts the parsed SVG into PDF and PNG formats. Finally, the wrapper function returns an _RLG Drawing_ object, which can be discarded.

<!-- ### Available functionality in the project package: 

1. All required functionality is in the package "project_utils/project1"
    
2. The initial seed can be obtained by the __get_seed()__ function
    
3. Each SVG can be converted into an [ElementTree](https://docs.python.org/3/library/xml.etree.elementtree.html) with the function __svg_to_tree(string)__. This functions returns the tree's root.

4. An [ElementTree](https://docs.python.org/3/library/xml.etree.elementtree.html) can be converted back into a string using __tree_to_string(root)__ function

5. The class [Coverage](notebooks/Coverage.ipynb) allows for coverage monitoring.

### Site notes:

1. Overall seed coverage is 4083 statements.
2. The __parse_svg(string)__ function has no error handling and any exception trown by the library will be propagated to the caller.
3. Mutations can be applied nodes or parts of a node. -->
<!-- 4. Tests reach an overall of 8498 unique statements -->

In [ ]:
import logging
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF, renderPM

# Required to run svglib on Python3
xrange = range

def parse_svg(data):
    print("Do something %s" % data[:20])
    with open("tmp.svg", "w") as f:
        f.write(data)
        f.close()
        
    logging.disable(logging.WARNING)
    drawing = svg2rlg("tmp.svg")
    logging.enable(logging.WARNING)

    assert(drawing is not None)

    pdf_file = 'tmp.pdf'
    png_file = 'tmp.png'    
    renderPDF.drawToFile(drawing, pdf_file)
    renderPM.drawToFile(drawing, png_file)
    return drawing

## Auxiliary functions

In order to aid the fuzzer's implementation we provide the following auxiliary functions:

In [ ]:
import sys
import os
from lxml import etree

def svg_as_tree(data):
    """Converts a String representation of an SVG into an ElementTree and returns its root

    :param data: String representation of an SVG
    :return: ElementTree https://docs.python.org/3/library/xml.etree.elementtree.html
    """
    parser = etree.XMLParser()
    root = etree.fromstring(data, parser=parser)
    return root


def tree_to_string(root):
    """Convert an SVG tree back into a string

    :param root: Root node of the SVG tree
    :return: String representation of the SVG
    """
    return etree.tostring(root)

## Fuzzer template

The basic template from our fuzzer is based on the [MutationCoverageFuzzer](MutationFuzzer.ipynb).

This template automatically loads a set of 21 SVG files as an initial seed.

In [ ]:
from Coverage import Coverage, population_coverage
from MutationFuzzer import MutationCoverageFuzzer, FunctionCoverageRunner

In [ ]:
class Project1MutationCoverageFuzzer(MutationCoverageFuzzer):
    def __init__(self, min_mutations=2, max_mutations=10):
        seed = self.__get_initial_seed()
        super(Project1MutationCoverageFuzzer, self).__init__(seed, min_mutations, max_mutations)

    def __get_initial_seed(self):
        """Gets the initial seed for the fuzzer

        :return: List of SVG in string format
        """

        seed_dir = "./data/svg/"
        seed_files = list(filter(lambda f: ".svg" in f, os.listdir(seed_dir)))

        seed = []
        for f in seed_files:
            with open(seed_dir + f) as x:
                s = ''.join(x.readlines()).strip()
                seed.append(s)

        print("Seed size: " + str(len(seed)) + " files")
        return seed

## Fuzzing the _svglib_

To fuzz _svglib_ it must execute it and inspect the obtained coverage. We will reuse the [FunctionCoverageRunner](MutationFuzzer.ipynb) class from the lecture to perform this task.

In [ ]:
parse_svg_runner = FunctionCoverageRunner(parse_svg)

We then initialize our fuzzer

In [ ]:
mutation_fuzzer = Project1MutationCoverageFuzzer()

And execute it multiple times to test it.

In [ ]:
mutation_fuzzer.reset()
mutation_fuzzer.runs(runner=parse_svg_runner, trials=10)
print(mutation_fuzzer.population)

## Obtaining the population coverage

In order to obtain the overal coverage achieved by the fuzzer's population we will exploit the [population_coverage](Coverage.ipynb) function from the lecture.

The following code calculates the overall coverage from a fuzzer's population:

In [ ]:
from Coverage import population_coverage
import matplotlib.pyplot as plt

In [ ]:
all_coverage, cumulative_coverage = population_coverage(
    mutation_fuzzer.population, parse_svg_runner)

plt.plot(cumulative_coverage)
plt.title('Coverage of urlparse() with random inputs')
plt.xlabel('# of inputs')
plt.ylabel('lines covered');